<a href="https://colab.research.google.com/github/sujathasivaraman/mlai/blob/main/chicken_cnn_binaryclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title RUN in GPU mode
# {"display-mode":"form", "form-width":"25%"}
#@markdown ##**BEFORE RUNNING ANY CODE, please change your Hardware Accelerator to GPU to train faster!**</h2>
#@markdown 1. Click on the **Runtime** menu at the top of the screen.
#@markdown 2. Click **Change Runtime Type**.
#@markdown 3. Choose **T4 GPU** under **Hardware Accelerator**.

#@markdown Once you've done that, run this code cell to check you're correctly connected!

import tensorflow as tf
from IPython.display import Markdown

if tf.test.gpu_device_name():
  display(Markdown("###✅ GPU connected!"))
else:
  display(Markdown("""
###❌ No GPU found!
If you're running into GPU limits when you try to switch, here are some suggestions:
  - Wait 12-24 hours for the limits to reset.
  - Share your copy of the notebook with another Google account that hasn't met the limit, and work through the notebook with that account.
  - Look into a paid subscription or paying for compute units as you go.
  """))

In [1]:
#@title Chickens Google Drive :  Run this to load images and imports! {"display-mode":"form", "form-width":"25%"}
import os
import zipfile
import matplotlib.pyplot as plt
import numpy as np  # Added for numerical operations

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Constants
IMG_SHAPE = (160, 160)  # Tuple format
BATCH_SIZE = 32  # More manageable batch size
zip_path = '/content/drive/My Drive/Dataset/chicken_notachicken.zip'
# extract_dir = '/content/drive/My Drive/Dataset/cats_and_dogs_filtered_extracted'
# zip_path = '/content/drive/My Drive/Dataset/cats_and_dogs_filtered.zip'
extract_dir = '/content/drive/MyDrive/Dataset'

# Create extraction directory
os.makedirs(extract_dir, exist_ok=True)  # Safer directory creation

# # Extract ZIP
# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_dir)

# Dataset directories
base_dir = os.path.join(extract_dir, 'chicken_notachicken')
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

print("Base directory:", base_dir)
print("Train Directory: ", train_dir)
print("Validation Directory: ", validation_dir)
print("Test Directory: ", test_dir)

# Verify directories
assert os.path.exists(train_dir), f"Train directory not found: {train_dir}"
assert os.path.exists(validation_dir), f"Validation directory not found: {validation_dir}"
assert os.path.exists(test_dir), f"Test directory not found: {test_dir}"

Mounted at /content/drive
Base directory: /content/drive/MyDrive/Dataset/chicken_notachicken
Train Directory:  /content/drive/MyDrive/Dataset/chicken_notachicken/train
Validation Directory:  /content/drive/MyDrive/Dataset/chicken_notachicken/validation
Test Directory:  /content/drive/MyDrive/Dataset/chicken_notachicken/test


In [2]:
#@title Chicken Keras create data generators with normalization
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
# Create data generators with normalization
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
#@title Chicken : Keras flow from directory : loading and augmenting image datasets
train_data = next(train_datagen.flow_from_directory(
    batch_size=76,
    directory=train_dir,
    shuffle=True,
    target_size=IMG_SHAPE, #(150,150)
    class_mode='binary')
)

validation_data = next(val_datagen.flow_from_directory(
    batch_size=5,
    directory=validation_dir,
    shuffle=False,
    target_size=IMG_SHAPE, #(150,150)
    class_mode='binary')
)

test_data = next(test_datagen.flow_from_directory(
    batch_size=20,
    directory=test_dir,
    shuffle=False,
    target_size=IMG_SHAPE, #(150,150)
    class_mode=None)
)

In [ ]:
#@title Chicken : train/validate/test
cd_train_inputs, cd_train_labels = train_data
cd_validation_inputs, cd_validation_labels = validation_data
# [the test data doesnt have two folders like chicken or notachicken it has only data folder]
cd_test_inputs= test_data
#cd_test_inputs, cd_test_labels = test_data

In [ ]:
#@title Chicken : Experiment ROC - MobileNetV2
from tensorflow.keras import layers, models, optimizers, metrics
from tensorflow.keras.applications import MobileNetV2
import tensorflow as tf

# Load pre-trained MobileNetV2 as feature extractor
pretrained_MobileNetV2model = MobileNetV2(
    input_shape=(160, 160, 3),       # your input image size
    include_top=False,               # exclude ImageNet classification head
    weights='imagenet',              # load pre-trained weights
    pooling=None                    # no pooling, we'll add it manually
)

# Freeze the MobileNetV2 layers for feature extraction (optional)
pretrained_MobileNetV2model.trainable = False

# Build the new model on top of MobileNetV2
modelMobileNetV2 = models.Sequential([
    pretrained_MobileNetV2model,         # Backbone
    layers.GlobalAveragePooling2D(),     # Reduces 4D tensor to 2D
    layers.Dropout(0.2),                 # Optional: helps prevent overfitting
#    layers.Dense(1, activation=None)     # Raw logits (use activation='sigmoid' if not using from_logits=True)
    layers.Dense(1, activation='sigmoid')     # Raw logits (use activation='sigmoid' if not using from_logits=True)
])
modelMobileNetV2.compile(
     optimizer=optimizers.Adamax(
         learning_rate=0.0001,       # Learning rate
         beta_1=0.9,                 # Exponential decay rate for the 1st moment estimates
         beta_2=0.999,               # Exponential decay rate for the 2nd moment estimates
         epsilon=1e-07,              # Small constant for numerical stability
         name='Adamax'
     ),
     loss=tf.keras.losses.BinaryCrossentropy(
         #from_logits=True,           # Set to True if last layer has no activation (e.g., no sigmoid)
         from_logits=False,           # Set to True if last layer has no activation (e.g., no sigmoid)
         label_smoothing=0.0         # Optional: smooth labels to prevent overconfidence
     ),
     metrics=[
         'accuracy',                            # Basic classification accuracy
         #metrics.Precision(name='precision'),   # Optional: precision metric
         #metrics.Recall(name='recall'),         # Optional: recall metric
         #metrics.AUC(name='auc'),               # Optional: area under ROC
     ],
     weighted_metrics=None,         # Optional: metrics evaluated with sample_weight
     run_eagerly=False,             # Run in eager mode (useful for debugging)
     steps_per_execution=1          # Steps per function call for performance tuning
 )

##############Optional if you are using testdata
#MobileNetV2_predictions = modelMobileNetV2.predict(test_data)

# pred_cat = []
# # for i in range(len(predictions)):
# #   print(predictions[i])

# for i in range(len(predictions)):
#   if predictions[i] < 0:
#     pred_cat.append("Chicken")
#   if predictions[i] > 0:
#     pred_cat.append("Not a Chicken")
#   print(predictions[i])
# pred_cat
###############################################

# Train the model
MobileNetV2_history = modelMobileNetV2.fit(
    cd_train_inputs,                     # Training input data (e.g., images)
    cd_train_labels,                     # Training labels (binary or one-hot)
    batch_size=32,                         # Number of samples per gradient update
    epochs=10,                             # Number of epochs to train
    verbose=1,                             # 0 = silent, 1 = progress bar, 2 = one line per epoch
    callbacks=None,                        # List of Keras callbacks (e.g., EarlyStopping, ModelCheckpoint)
    #validation_split=0.0,                  # Use this % of training data for validation (instead of validation_data)
    validation_data=(cd_validation_inputs, cd_validation_labels),  # Data on which to evaluate the loss and metrics
    #validation_data=(cd_test_inputs, abs(MobileNetV2_predictions)),  # Data on which to evaluate the loss and metrics
    shuffle=True,                          # Whether to shuffle training data before each epoch
    class_weight=None,                     # Optional dict mapping class indices to weights (e.g., {0: 1.0, 1: 3.0})
    sample_weight=None,                    # Optional array of weights for individual training samples
    initial_epoch=0,                       # Epoch at which to start training (useful when resuming)
    steps_per_epoch=None,                 # Use for generators/datasets; otherwise leave as None
    validation_steps=None,                # Same as above for validation set
    validation_batch_size=None,           # Batch size for validation data
    validation_freq=1,                    # Evaluate validation data every n epochs
    #max_queue_size=10,                    # Generator prefetch queue
    #workers=1,                             # Number of CPU threads for generators
    #use_multiprocessing=False             # Use multiprocessing when using data generators
)

#@title Chicken : Accuracy and Loss for modelhistory (MobileNetV2, EfficientB0, VGG16)
acc = mobilenetv2_history .history['accuracy']
val_acc = mobilenetv2_history.history['val_accuracy']
loss = mobilenetv2_history.history['loss']
val_loss = mobilenetv2_history.history['val_loss']

############################################################

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend(loc='lower right')
plt.ylim([min(plt.ylim()),1])

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()
############################################################

# After training, generate predictions (example for ROC curve calculation)
MobileNetV2_predictions = modelMobileNetV2.predict(cd_validation_inputs)
#curve_prediction = tf.sigmoid(MobileNetV2_predictions).numpy()
curve_prediction= MobileNetV2_predictions

plt.figure(figsize=(8, 8))
print(len(test_data))
for i in range(min(9, len(test_data))): # Limit the loop to a maximum of 9 images
    plt.subplot(3, 3, i+1)
    plt.imshow(test_data[i])
#    plt.title(f"Label: {pred_cat[i]}({predictions[i]})")
    plt.title(f"Label: {curve_prediction[i]}")
    plt.axis('off')
plt.show()

In [ ]:
#@title Experiment accept image and then predict
# First we need to upload the image
from google.colab import files
from IPython.display import Image, display
import os

# Upload the image file
uploaded = files.upload()
predicted_label=""

# Get the uploaded filename
img_name = next(iter(uploaded))
 # Use your actual model variable
# Now predict using the model
predicted_label = predict_image(modelMobileNetV2, img_name)
#predicted_label = predict_image(modelvgg16, img_name)

print(f"Sujatha : The model predicts that the image is a {predicted_label}")

# Display the image
display(Image(img_name))

# Clean up the uploaded file
os.remove(img_name)

#@title Experiment: Upload Image and Predict
# # Step 1: Import Necessary Libraries
# from google.colab import files
from IPython.display import Image, display, clear_output
# import ipywidgets as widgets
import numpy as np
from tensorflow.keras.preprocessing import image
# import os

# # Define image shape expected by your model
# IMG_SHAPE = (150, 150)  # Update this to match your model's input shape

# # Step 2: Create an Upload Button
# uploader = widgets.FileUpload(
#     accept='image/*',  # Accept only image files
#     multiple=False  # Do not allow multiple files
# )
# display(uploader)

# # Define the Prediction Function


In [ ]:
#@title experiment TPR, NPR, Threshold, ROC, AUC using pretrained models VGG16, MobileNet, EfficientnetB0
#Add ROC & AUC Plotting for Binary Classifier

from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np

# Apply sigmoid if predictions are raw logits
prob_predictions = tf.sigmoid(curve_prediction).numpy()
#Krithik exercise for you, generate ROC for testdata using your prediction label array as test labels.

fpr, tpr, thresholds = roc_curve(cd_validation_labels, prob_predictions)
#fpr, tpr, thresholds = roc_curve(cd_validation_labels, vgg16_predictions)
#fpr, tpr, thresholds = roc_curve(cd_validation_labels, mobilenetv2_predictions)

roc_auc = auc(fpr, tpr)

# Plot ROC
plt.figure(figsize=(6,6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=1, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for Chicken Classifier')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()


